# Importing Libraries 

In [188]:
from __future__ import division
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np

# Defining non-linearities

In [189]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def ReLU(x):
    shape = x.shape
    x = x.reshape(-1)
    null = np.argwhere(x < 0)
    x[null] = 0
    x = x.reshape(shape)
    return x

def softmax(x):
    d = np.exp(x).sum()
    return np.exp(x)/d

# Convolution Layer of CNN

In [190]:
def conv_layer(inputs, kernels, kernel_size, stride, padding, activation, kernel_weights):
    [n_r,n_c,d] = inputs.shape
    #number of filters
    F = kernels
    [f_r,f_c] = kernel_size
    bias = np.random.normal(0,1,kernels)
    if padding == "same":
        out = np.zeros(((int)(n_r/stride),(int)(n_c/stride),F))
        #p_r == # of layers to be padded.
        [p_r,p_c] = ((int)((stride*(n_r/stride-1) + f_r - n_r)/2),(int)((stride*(n_c/stride-1) + f_c - n_c)/2))
        inp_padded = np.zeros((n_r+2*p_r,n_c+2*p_c,d))
        #r == row step, c == column step
        [r,c] = (np.arange(0,n_r+2*p_r-f_r+1,stride),np.arange(0,n_c+2*p_c-f_c+1,stride))
        #transforming input into its padded version
        for i in range (p_r,n_r+p_r):
            for j in range (p_c,n_c+p_c):
                for k in range (0,d):
                    inp_padded[i][j][k] = inputs[i-p_r][j-p_c][k]
    if padding == "valid":
        out = np.zeros(((int)((n_r - f_r)/stride) + 1, (int)((n_c - f_c)/stride) + 1,F), dtype=float)
        [r,c] = (np.arange(0,n_r-f_r+1,stride),np.arange(0,n_c-f_r+1,stride))
    #chunck from input volume
    A = np.zeros((f_r,f_c,d))
    sub_ker = np.zeros((f_r,f_c,d))
    #performing convolution             
    for i in r:
        for j in c:
            for k in range(0,d):
                for k_r in range(i,i+f_r):
                    for k_c in range(j, j+f_c):
                        if padding == "same":
                            A[k_r-i][k_c-j][k] = inp_padded[k_r][k_c][k]
                        if padding == "valid":
                            A[k_r-i][k_c-j][k] = inputs[k_r][k_c][k]
            for f in range(0,F):
                for l in range(0,d):
                    for m in range(0,f_r):
                        for n in range(0,f_c):
                            sub_ker[m][n][l] = kernel_weights[f][m][n][l]
                out[np.asscalar(np.argwhere(r == i))][np.asscalar(np.argwhere(c == j))][f] = (np.multiply(sub_ker,A)).sum() + np.asscalar(bias[f])  
    out_unactivated = out
    if activation == "sigmoid":
        out = sigmoid(out)
    if activation == "tanh":
        out = np.tanh(out)
    if activation == "ReLU":
        out = ReLU(out)
    return out,out_unactivated,inp_padded

# Pooling Layer of CNN

In [191]:
def pooling_function(c_out, stride, pool_type):
    [n_r,n_c,d] = c_out.shape
    p_out = np.zeros(((int)(n_r/stride), (int)(n_c/stride),d), dtype=float)
    #r = row step, c = column step
    [r,c] = (np.arange(0,n_r-stride+1,stride),np.arange(0,n_c-stride+1,stride))
    A = np.zeros((stride,stride))
    #input type matrix with ones at pooled outputs and zeros at non-pooled items
    pooled_values = np.zeros(c_out.shape)
    for i in r:
        for j in c:
            for k in range(0,d):
                for k_r in range(i,i+stride):
                    for k_c in range(j, j+stride):
                        A[k_r-i][k_c-j] = c_out[k_r][k_c][k]
                if pool_type == "max":
                    pooled_values[np.argwhere(A == np.max(A))[0][0]+i][np.argwhere(A == np.max(A))[0][1]+j][k] = 1
                    p_out[np.asscalar(np.argwhere(r == i))][np.asscalar(np.argwhere(c == j))][k] =  np.max(A)
                if pool_type == "average":
                    p_out[np.asscalar(np.argwhere(r == i))][np.asscalar(np.argwhere(c == j))][k] =  np.mean(A.reshape(-1))
                if pool_type == "l2norm":
                    p_out[np.asscalar(np.argwhere(r == i))][np.asscalar(np.argwhere(c == j))][k] =  np.linalg.norm(A)
    return p_out,pooled_values

# MNIST Data

In [192]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train = np.vstack([img.reshape((28, 28)) for img in mnist.train.images])
y_train = mnist.train.labels
X_test = np.vstack([img.reshape(28, 28) for img in mnist.test.images])
y_test = mnist.test.labels
del mnist

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [193]:
X_train = X_train.reshape((-1,28,28,1))
X_test = X_test.reshape((-1,28,28,1))

# CNN Architecture

In [194]:
conv_layers = 2
kernels = [8,16]
kernel_sizes = [(5,5),(5,5)]
pool_type = "max"
pool_size = [(2,2),(2,2)]
pool_strides = [2,2]
cnn_activations = ["ReLU","ReLU"]
dense_units = 1024
logit_units = 10
r = 0.01

In [195]:
def relu_derivative(x):
    a = np.argwhere(x > 0)
    b = np.argwhere(x <= 0)
    for i in range(0,a.shape[0]):
        x[a[i][0]][a[i][1]][a[i][2]] = 1
    for i in range(0,b.shape[0]):
        x[b[i][0]][b[i][1]][b[i][2]] = 1
    return x

In [196]:
def convolve2D(inputs,filters):
    out = np.zeros((filters.shape[2],inputs.shape[0]-filters.shape[0]+1,inputs.shape[1]-filters.shape[1]+1,inputs.shape[2]))
    A = np.zeros((filters.shape[0],filters.shape[1]))
    sub_ker = np.zeros(A.shape)
    for i in range(0,filters.shape[2]):
        for r_i in range(0,filters.shape[0]):
            for c_i in range(0,filters.shape[1]):
                sub_ker[r_i][c_i] = filters[r_i][c_i][i]
        for j in range(0,inputs.shape[2]):
            for k_i in range(0,out.shape[1]):
                    for k_j in range(0,out.shape[2]):
                        for k_r in range(k_i,k_i+filters.shape[0]):
                            for k_c in range(k_j,k_j+filters.shape[1]):
                                A[k_r-k_i][k_c-k_j] = inputs[k_r][k_c][j]
                        out[i][k_i][k_j][j] = np.multiply(sub_ker,A).sum()
    return out

In [197]:
def convolve3D(inputs,filters):
    out = np.zeros((inputs.shape[0],inputs.shape[1],filters.shape[3]))
    input_padded = np.zeros((inputs.shape[0]+filters.shape[1]-1,inputs.shape[1]+filters.shape[2]-1,inputs.shape[2]))
    [p_r,p_c] = [(int)((filters.shape[1]-1)/2),(int)((filters.shape[2]-1)/2)]
    for k in range(0,inputs.shape[2]):
        for i in range(p_r,input_padded.shape[0]-p_r):
            for j in range(p_c,input_padded.shape[1]-p_c):
                input_padded[i][j][k] = inputs[i-p_r][j-p_c][k]
    A = np.zeros((filters.shape[1],filters.shape[2],filters.shape[0]))
    sub_ker = np.zeros(A.shape)
    for k in range(0,filters.shape[3]):
        for i in range(0,filters.shape[1]):
            for j in range(0,filters.shape[2]):
                for d in range(0,filters.shape[0]):
                    sub_ker[i][j][d] = filters[d][i][j][k]
        for i in range(0,out.shape[0]):
            for j in range(0,out.shape[1]):
                for r_i in range(i,i+filters.shape[1]):
                    for r_j in range(j,j+filters.shape[2]):
                        for d in range(0,filters.shape[0]):
                            A[r_i-i][r_j-j][d] = input_padded[r_i][r_j][d]
                out[i][j][k] = np.multiply(A,sub_ker).sum()
    return out

In [198]:
def convolve(inputs,filters,kernel_size):
    inputs = inputs.reshape((28,28))
    input_padded = np.zeros((28+kernel_size[0]-1,28+kernel_size[1]-1))
    sub_ker = np.zeros((28,28))
    A = np.zeros((28,28))
    out = np.zeros((filters.shape[2],kernel_size[0],kernel_size[1]))
    for i in range((int)((kernel_size[0]-1)/2),input_padded.shape[0]-(int)((kernel_size[0]-1)/2)):
            for j in range((int)((kernel_size[1]-1)/2),input_padded.shape[1]-(int)((kernel_size[1]-1)/2)):
                input_padded[i][j] = inputs[i-(int)((kernel_size[0]-1)/2)][j-(int)((kernel_size[0]-1)/2)]
    for k in range(0,filters.shape[2]):
        for i in range(0,28):
            for j in range(0,28):
                sub_ker[i][j] = filters[i][j][k]
        for i in range(0,kernel_size[0]):
            for j in range(0,kernel_size[1]):
                for r_i in range(i,i+28):
                    for r_j in range(j,j+28):
                        A[r_i-i][r_j-j] = input_padded[r_i][r_j]
                out[k][i][j] = np.multiply(A,sub_ker).sum()
    return out.reshape((out.shape[0],out.shape[1],out.shape[2],1))

# Batch Gradient Descent

In [199]:
#weight initialization
w_conv2D1 = np.random.normal(0,1,(kernels[0],kernel_sizes[0][0],kernel_sizes[0][1],1))
w_conv2D1 = (1/np.sqrt(np.size(w_conv2D1)/kernels[0]))*w_conv2D1
w_conv2D2 = np.random.normal(0,1,(kernels[1],kernel_sizes[1][0],kernel_sizes[1][1],kernels[0]))
w_conv2D2 = (1/np.sqrt(np.size(w_conv2D2)/kernels[1]))*w_conv2D2
w_dense = np.random.normal(0,1,(7*7*kernels[1],dense_units))
w_dense = (1/np.sqrt(7*7*kernels[1]))*w_dense
w_logit = np.random.normal(0,1,(dense_units,logit_units))
w_logit = (1/np.sqrt(dense_units))*w_logit

for epochs in range(0,15):
    batch_size = 55
    
    #shuffling the train_images
    shuffle = np.arange(X_train.shape[0])
    np.random.shuffle(shuffle)
    c = 0
    
    for i in shuffle:
        X_train[c],X_train[i] = X_train[i],X_train[c]
        y_train[c],y_train[i] = y_train[i],y_train[c]
        c += 1
    batches = X_train.reshape((batch_size,(int)(X_train.shape[0]/batch_size),X_train.shape[1],X_train.shape[2],X_train.shape[3]))
    
    #takes sum of gradients in a batch
    w_logit_g_sum = np.zeros((dense_units, logit_units))
    w_dense_g_sum = np.zeros((7*7*kernels[1], dense_units))
    w_conv2D2_g_sum = np.zeros((kernels[1], kernel_sizes[1][0], kernel_sizes[1][1], kernels[0]))
    w_conv2D1_g_sum = np.zeros((kernels[0], kernel_sizes[0][0], kernel_sizes[0][1], 1))
    
    for i in range(0,(int)(X_train.shape[0]/batch_size)):
        for j in range(0,batch_size):
            image = batches[i][j]
            
            #forward pass
            
            #convolution layer - 1
            [out_conv2D1,out_conv2D1_unactivated,inp_conv2D1_padded] = conv_layer(image, kernels[0], kernel_sizes[0], 1, "same", cnn_activations[0],w_conv2D1)
            
            #pool layer - 1 
            [pool_out1,pooled_values1] = pooling_function(out_conv2D1, pool_strides[0], pool_type)
            
            #convolution layer - 2
            [out_conv2D2,out_conv2D2_unactivated,inp_conv2D2_padded] = conv_layer(pool_out1, kernels[1], kernel_sizes[1], 1, "same", cnn_activations[1],w_conv2D2)
            
            #pool_layer - 2
            [pool_out2,pooled_values2] = pooling_function(out_conv2D2, pool_strides[1], pool_type)
            
            #flattening pool_layer output
            flatten = pool_out2.reshape(-1)
            
            #dense layer nodes
            dense_out = np.dot(flatten,w_dense)
            
            #out
            logit_out = softmax(np.dot(dense_out,w_logit))
            
            error = y_train[i*batch_size+j] - logit_out
            
            #finding gradients with a backward pass
            
            #gradients of weights to logit layer from dense layer
            w_logit_g = np.asmatrix(dense_out).T*np.asmatrix(error)
            w_logit_g_sum += w_logit_g
            
            #gradients of weights to dense layer from flatten layer
            w_dense_g = np.asmatrix(flatten).T*(np.asmatrix(error)*w_logit_g.T)
            w_dense_g_sum += w_dense_g
            
            w_mlp_g = w_dense_g*(np.asmatrix(error)*w_logit_g.T).T
            w_mlp_g = np.asarray(w_mlp_g)
            
            #transforming gradients behind pool layer - 2
            w_g_afterpool2 = w_mlp_g.reshape((7,7,kernels[1]))
            w_g_pool2 = np.multiply(pooled_values2,np.repeat(np.repeat(w_g_afterpool2,2,axis=1),2,axis=0))
            w_g_filters2 = np.multiply(w_g_pool2,relu_derivative(out_conv2D2_unactivated))
            
            #gradients of weights in convolution layer - 2
            w_conv2D2_g = convolve2D(np.asarray(inp_conv2D2_padded),np.asarray(w_g_filters2))
            w_conv2D2_g_sum += w_conv2D2_g
            
            #transforming gradients behind pool layer - 1
            w_g_afterpool1 = convolve3D(np.asarray(w_g_filters2),np.asarray(w_conv2D2_g))
            w_g_pool1 = np.multiply(pooled_values1,np.repeat(np.repeat(w_g_afterpool1,2,axis=1),2,axis=0))
            w_g_filters1 = np.multiply(w_g_pool1,relu_derivative(out_conv2D1_unactivated))
            
            #gradients of weights in convolution layer - 1
            w_conv2D1_g = convolve(image,w_g_filters1,kernel_sizes[0])
            w_conv2D1_g_sum += w_conv2D1_g
        #updating weights with batch gradients    
        w_conv2D1 -= r*(1/batch_size)*w_conv2D1_g_sum
        w_conv2D2 -= r*(1/batch_size)*w_conv2D2_g_sum
        w_dense -= r*(1/batch_size)*w_dense_g_sum
        w_logit -= r*(1/batch_size)*w_logit_g_sum

# Stochastic Gradient Descent

In [202]:
w_conv2D1 = np.random.normal(0,1,(kernels[0],kernel_sizes[0][0],kernel_sizes[0][1],1))
w_conv2D1 = (1/np.sqrt(np.size(w_conv2D1)/kernels[0]))*w_conv2D1
w_conv2D2 = np.random.normal(0,1,(kernels[1],kernel_sizes[1][0],kernel_sizes[1][1],kernels[0]))
w_conv2D2 = (1/np.sqrt(np.size(w_conv2D2)/kernels[1]))*w_conv2D2
w_dense = np.random.normal(0,1,(7*7*kernels[1],dense_units))
w_dense = (1/np.sqrt(7*7*kernels[1]))*w_dense
w_logit = np.random.normal(0,1,(dense_units,logit_units))
w_logit = (1/np.sqrt(dense_units))*w_logit

for epochs in range(0,15):
    
    shuffle = np.arange(X_train.shape[0])
    np.random.shuffle(shuffle)
    train_error = np.zeros(15)
    
    for i in shuffle:
        image = X_train[i]
        
        #forward pass
        [out_conv2D1,out_conv2D1_unactivated,inp_conv2D1_padded] = conv_layer(image, kernels[0], kernel_sizes[0], 1, "same", cnn_activations[0],w_conv2D1)
        
        [pool_out1,pooled_values1] = pooling_function(out_conv2D1, pool_strides[0], pool_type)
        
        [out_conv2D2,out_conv2D2_unactivated,inp_conv2D2_padded] = conv_layer(pool_out1, kernels[1], kernel_sizes[1], 1, "same", cnn_activations[1],w_conv2D2)
        
        [pool_out2,pooled_values2] = pooling_function(out_conv2D2, pool_strides[1], pool_type)
        
        flatten = pool_out2.reshape(-1)
        
        dense_out = np.dot(flatten,w_dense)
        
        logit_out = softmax(np.dot(dense_out,w_logit))
        
        error = y_train[i] - logit_out
        
        #finding gradients with a backward pass
        w_logit_g = np.asmatrix(dense_out).T*np.asmatrix(error)
        
        w_dense_g = np.asmatrix(flatten).T*(np.asmatrix(error)*w_logit_g.T)
        
        w_mlp_g = w_dense_g*(np.asmatrix(error)*w_logit_g.T).T
        w_mlp_g = np.asarray(w_mlp_g)
        
        w_g_afterpool2 = w_mlp_g.reshape((7,7,kernels[1]))
        w_g_pool2 = np.multiply(pooled_values2,np.repeat(np.repeat(w_g_afterpool2,2,axis=1),2,axis=0))
        w_g_filters2 = np.multiply(w_g_pool2,relu_derivative(out_conv2D2_unactivated))
        
        w_conv2D2_g = convolve2D(np.asarray(inp_conv2D2_padded),np.asarray(w_g_filters2))
        
        w_g_afterpool1 = convolve3D(np.asarray(w_g_filters2),np.asarray(w_conv2D2_g))
        w_g_pool1 = np.multiply(pooled_values1,np.repeat(np.repeat(w_g_afterpool1,2,axis=1),2,axis=0))
        w_g_filters1 = np.multiply(w_g_pool1,relu_derivative(out_conv2D1_unactivated))
        
        w_conv2D1_g = convolve(image,w_g_filters1,kernel_sizes[0])
        
        w_conv2D1 -= r*w_conv2D1_g
        w_conv2D2 -= r*w_conv2D2_g
        w_dense -= r*w_dense_g 
        w_logit -= r*w_logit_g
        
        train_error[epochs] += (-1*(np.multiply(y_train[i],np.log(logit_out)).sum()))